In [75]:
#Inicializacion de spark session y creacion de nuestra instancia
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField,StructType,IntegerType,StringType,FloatType

spark = SparkSession.builder.appName('sesion3').getOrCreate()

In [76]:
#Para trabajar con estas funciones tenemos que importarlas de la siguiente forma
from pyspark.sql.functions import countDistinct,avg,stddev


In [77]:
nombres = spark.read.csv('nombres_comunes.csv',inferSchema=True,header=True)

In [78]:
nombres.printSchema()

root
 |-- NOMBRE: string (nullable = true)
 |-- TOTAL: integer (nullable = true)
 |-- CADA_100: double (nullable = true)
 |-- POBLACION: string (nullable = true)
 |-- SEXO: string (nullable = true)



In [79]:
#Si no nos acaba de gustar el formato podemos asignar el que nosotros queramos como ya hemos visto antes,
#en este caso me interesa poner que los numeros son numeros ya que vamos a trabajar con ellos:
mi_estructura = [StructField('Nombre',StringType(),False),
              StructField('Total',IntegerType(),True),
              StructField('Cada_100',FloatType(),False),
                 StructField('Poblacion',StringType(),False),
                 StructField('Sexo',StringType(),False),
             ]
mi_esquema = StructType(fields=mi_estructura)

nombres = spark.read.csv('nombres_comunes.csv',inferSchema=True,header=True,schema=mi_esquema)
nombres.printSchema()

root
 |-- Nombre: string (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Cada_100: float (nullable = true)
 |-- Poblacion: string (nullable = true)
 |-- Sexo: string (nullable = true)



In [80]:
nombres.show()

+----------------+-----+--------+---------+------+
|          Nombre|Total|Cada_100|Poblacion|  Sexo|
+----------------+-----+--------+---------+------+
|         ANTONIO|28849|    31.3| ALICANTE|HOMBRE|
|            JOSE|26535|    28.8| ALICANTE|HOMBRE|
|       FRANCISCO|23354|    25.4| ALICANTE|HOMBRE|
|          MANUEL|19218|    20.9| ALICANTE|HOMBRE|
|    JOSE ANTONIO|13145|    14.3| ALICANTE|HOMBRE|
|           DAVID|13045|    14.2| ALICANTE|HOMBRE|
|            JUAN|12590|    13.7| ALICANTE|HOMBRE|
|       ALEJANDRO|10813|    11.7| ALICANTE|HOMBRE|
|         VICENTE|10806|    11.7| ALICANTE|HOMBRE|
|          JAVIER|10545|    11.5| ALICANTE|HOMBRE|
|          DANIEL|10481|    11.4| ALICANTE|HOMBRE|
|          CARLOS| 9977|    10.8| ALICANTE|HOMBRE|
|          MIGUEL| 9161|     9.9| ALICANTE|HOMBRE|
|          RAFAEL| 9112|     9.9| ALICANTE|HOMBRE|
|           PEDRO| 9062|     9.8| ALICANTE|HOMBRE|
|     JOSE MANUEL| 8672|     9.4| ALICANTE|HOMBRE|
|       JOSE LUIS| 8669|     9.

# COUNTDISTINCT()
En PySpark, podemos utilizar la función distinct().count() para obtener un recuento de elementos distintos,
distinct() elimina registros duplicados (que coinciden con todas las columnas de una fila) 
de DataFrame, count () devuelve el recuento de registros del DataFrame. 
Al encadenarlos, podemos obtener el recuento de elementos distintos del DataFrame.

CountDistinct() es una función SQL que podría usarse para obtener el recuento distinto de las columnas seleccionadas.



In [81]:
#Si queremos ver el numero de poblaciones distintas que tenemos en nuestro dataset, hariamos lo siguiente:
nombres.select(countDistinct('POBLACION')).show()

+-------------------------+
|count(DISTINCT POBLACION)|
+-------------------------+
|                        2|
+-------------------------+



# ALIAS()
Con alias() podemos asignar nombres a la columna, como vemos en el ejemplo anterior, el nombre de la columna
que nos pone por defecto es count(Distinct poblacion), a lo mejor nos interesa llamarle por ejemplo total_poblaciones
para esto usaremos alias()



In [82]:
nombres.select(countDistinct('POBLACION').alias('Total_Poblaciones')).show()

+-----------------+
|Total_Poblaciones|
+-----------------+
|                2|
+-----------------+



# AVG()
Con AVG() vamos a sacar la media de la columna que queramos

In [83]:
nombres.select(avg('TOTAL')).show()

+-----------------+
|       avg(TOTAL)|
+-----------------+
|8976.604938271605|
+-----------------+



# STDDEV()
Usaremos esta funcion para sacar la desviacion estandar 
(Una desviación estándar baja indica que la mayor parte de los datos de una muestra tienden a estar agrupados cerca de su media)

In [84]:
nombres.select(stddev('TOTAL')).show()

+------------------+
|stddev_samp(TOTAL)|
+------------------+
| 7825.255247842623|
+------------------+



# FormatNumber()
Como vemos cuando hacemos medias, o este tipo de operaciones nos devuleve un decimal muy largo, en este caso a lo mejor 
nos puede interesar formatear el valor y limpiar decimales por ejemplo


In [85]:
#Por ejemplo para la media, queremos solo dos decimales
nombres.select(avg('TOTAL').alias('Media')).show()

+-----------------+
|            Media|
+-----------------+
|8976.604938271605|
+-----------------+



In [86]:
from pyspark.sql.functions import format_number

In [87]:
media = nombres.select(avg('TOTAL').alias('Media'))

In [88]:
#Le pasamos a formar number la columna y los decimales que queremos que saque
media.select(format_number('Media',2).alias('MEDIA_2_DECIMALES')).show()

+-----------------+
|MEDIA_2_DECIMALES|
+-----------------+
|         8,976.60|
+-----------------+



# OrderBY()
#Sirve para ordenar en base a una columna, ascendente o descedente


In [89]:
nombres.orderBy('TOTAL').show()

+----------------+-----+--------+---------+------+
|          Nombre|Total|Cada_100|Poblacion|  Sexo|
+----------------+-----+--------+---------+------+
|        SANTIAGO| 3395|     3.7| ALICANTE|HOMBRE|
|      INMACULADA| 3464|     3.7| ALICANTE| MUJER|
|     JUAN MANUEL| 3471|     3.8| ALICANTE|HOMBRE|
|           NURIA| 3535|     3.8| ALICANTE| MUJER|
|     MARIA JESUS| 3543|    3.84| ALICANTE| MUJER|
|FRANCISCO MANUEL| 3623|     3.8|  SEVILLA|HOMBRE|
|         ROBERTO| 3676|     4.2| ALICANTE|HOMBRE|
|          ALICIA| 3676|     3.9| ALICANTE| MUJER|
|           MARIO| 3678|     4.1| ALICANTE|HOMBRE|
|         BEATRIZ| 3735|     4.1| ALICANTE| MUJER|
|   ANTONIO JESUS| 3737|     3.9|  SEVILLA|HOMBRE|
|           JULIA| 3742|     4.1| ALICANTE| MUJER|
|           DIEGO| 3744|     4.1| ALICANTE|HOMBRE|
|        SALVADOR| 3788|    4.01|  SEVILLA|HOMBRE|
|          VICTOR| 3799|     4.1| ALICANTE|HOMBRE|
|        VERONICA| 3845|     4.1| ALICANTE| MUJER|
|        MERCEDES| 3850|     4.

In [90]:
#Para cambiarlo a descendente por ejemplo

nombres.orderBy(nombres['TOTAL'].desc()).show()

+----------------+-----+--------+---------+------+
|          Nombre|Total|Cada_100|Poblacion|  Sexo|
+----------------+-----+--------+---------+------+
|          MANUEL|59967|    63.2|  SEVILLA|HOMBRE|
|         ANTONIO|51883|    54.7|  SEVILLA|HOMBRE|
|            JOSE|36789|    38.8|  SEVILLA|HOMBRE|
|       FRANCISCO|32232|    34.1|  SEVILLA|HOMBRE|
|         ANTONIO|28849|    31.3| ALICANTE|HOMBRE|
|            JOSE|26535|    28.8| ALICANTE|HOMBRE|
|       FRANCISCO|23354|    25.4| ALICANTE|HOMBRE|
|           MARIA|23073|    24.6| ALICANTE| MUJER|
|FRANCISCO JAVIER|22280|    23.5|  SEVILLA|HOMBRE|
|    MARIA CARMEN|22143|    23.6| ALICANTE| MUJER|
|          RAFAEL|21640|    22.8|  SEVILLA|HOMBRE|
|     JOSE MANUEL|21123|    22.3|  SEVILLA|HOMBRE|
|    JOSE ANTONIO|20725|    21.8|  SEVILLA|HOMBRE|
|            JUAN|19546|    20.6|  SEVILLA|HOMBRE|
|          MANUEL|19218|    20.9| ALICANTE|HOMBRE|
|          JOSEFA|17616|    18.8| ALICANTE| MUJER|
|           JESUS|15693|    16.

In [91]:
nombres.orderBy(nombres['TOTAL'].asc()).show()

+----------------+-----+--------+---------+------+
|          Nombre|Total|Cada_100|Poblacion|  Sexo|
+----------------+-----+--------+---------+------+
|        SANTIAGO| 3395|     3.7| ALICANTE|HOMBRE|
|      INMACULADA| 3464|     3.7| ALICANTE| MUJER|
|     JUAN MANUEL| 3471|     3.8| ALICANTE|HOMBRE|
|           NURIA| 3535|     3.8| ALICANTE| MUJER|
|     MARIA JESUS| 3543|    3.84| ALICANTE| MUJER|
|FRANCISCO MANUEL| 3623|     3.8|  SEVILLA|HOMBRE|
|         ROBERTO| 3676|     4.2| ALICANTE|HOMBRE|
|          ALICIA| 3676|     3.9| ALICANTE| MUJER|
|           MARIO| 3678|     4.1| ALICANTE|HOMBRE|
|         BEATRIZ| 3735|     4.1| ALICANTE| MUJER|
|   ANTONIO JESUS| 3737|     3.9|  SEVILLA|HOMBRE|
|           JULIA| 3742|     4.1| ALICANTE| MUJER|
|           DIEGO| 3744|     4.1| ALICANTE|HOMBRE|
|        SALVADOR| 3788|    4.01|  SEVILLA|HOMBRE|
|          VICTOR| 3799|     4.1| ALICANTE|HOMBRE|
|        VERONICA| 3845|     4.1| ALICANTE| MUJER|
|        MERCEDES| 3850|     4.